In [1]:
import sys
sys.path.append('/home/mourad/DS/Mosmovi/Mosmovi/')
import pandas as pd
import os
import numpy as np
import multiprocessing as mp
import seaborn as sns
import matplotlib.pyplot as plt
import random

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

pd.set_option('max_columns', None)
pd.set_option('display.max_rows', 100)
from collections import Counter

import unicodedata
import importlib

from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', lambda x: '%.18f' % x)

In [2]:
def format_csvs_to_df(path):
    file_names = os.listdir(path)
    frames = []
    for n in file_names:
        coordinates = [float(x) for x in n[:-4].split('_')]
        df = pd.read_csv(path + n, sep=';')
        df['coordinates'] = [coordinates] * len(df)
        frames.append(df)
    df_total = pd.concat(frames)
    return df_total

def train_test_split_by_users(df, percentage_users = 0.10):
    """
    Take a dataframe in input and return train and test sets. 
    """
    users_ids = df.author_id
    users_ids = set(df.author_id)
    n_samples = int(len(users_ids)*percentage_users)
    val_users = random.sample(users_ids, n_samples)
    ind = df.author_id.isin(val_users)
    return df.loc[~ind], df.loc[ind]

In [4]:
%%time
df = format_csvs_to_df('../../data/Copy of dev_south_america_merged/')

Wall time: 2min 23s


In [5]:
%%time
#Convert to datetime
df['created_at'] = pd.to_datetime(df['created_at'])
#Extract lat and long
df['lat'], df['long'] = df.coordinates.apply(lambda x : x[0]), df.coordinates.apply(lambda x : x[1])
#Avoid scientific notation for ID type
df['id'] = df['id'].astype(np.int64).astype(str)
df['conversation_id'] = df['conversation_id'].fillna(0).astype(np.int64).astype(str)
df['in_reply_to_user_id'] = df['in_reply_to_user_id'].fillna(0).astype(np.int64).astype(str)
df['author_id'] = df['author_id'].fillna(0).astype(np.int64).astype(str)
#Convert coordinates to strings to make dupli analysis
df['coordinates'] = df['coordinates'].astype(str)


Wall time: 58.7 s


In [5]:
%%time
df.to_csv('../../data/formated_data_w_longlat.csv')

Wall time: 5min 29s


In [6]:
tocheckdupli = [ 'id', 'text','geo_geo_bbox', 'author_id', 'coordinates']

In [7]:
df_clean = df.drop_duplicates(subset=tocheckdupli)

In [10]:
df_te = df.iloc[:1000]

In [31]:
df.id.value_counts()

1363508983810637828    8
1364445644740558854    8
1364062108627927042    8
1362007744694390789    8
1364440744690335744    8
                      ..
1422677229641814033    1
1430989639771361283    1
1429077363552395266    1
1427349363764891649    1
1432116311795113985    1
Name: id, Length: 6799149, dtype: int64

In [27]:
s = df[['id','coordinates']].groupby('id').nunique()
# df1 = s['coordinates' > 1].reset_index()
s.loc[s.coordinates>1].shape[0]

2066947

In [29]:
s = df[['id','text']].groupby('id').nunique()
# df1 = s['coordinates' > 1].reset_index()
s.loc[s.text>1].shape[0]

95

In [30]:
s = df[['id','geo_geo_bbox']].groupby('id').nunique()
# df1 = s['coordinates' > 1].reset_index()
s.loc[s.geo_geo_bbox>1].shape[0]

3

### Exploration:

In [13]:
test = df.loc[df['id'] == '1357494046369972224']
test.coordinates = test.coordinates.astype(str) 
print(set(test.text))
print(len(set(test.coordinates.astype(str))))
# test.groupby('text').nunique()[['coordinates']]
test.loc[test.text=='Ya me toca comprar otros briefs jajaja 😅.']

{'Ya me toca comprar otros briefs jajaja 😅.', 'Representación exacta de uribistas y duquistas al tocar un libro. https://t.co/NhsUMC7jsN'}
6


/home/mourad/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,lang,id,created_at,text,conversation_id,in_reply_to_user_id,reply_settings,source,possibly_sensitive,geo_place_id,geo_place_type,geo_country,geo_full_name,geo_country_code,geo_geo_type,geo_geo_bbox,geo_geo_properties,author_id,author_profile_image_url,author_username,author_url,author_name,author_created_at,author_verified,author_protected,author_description,retweet_count,reply_count,like_count,quote_count,followers_count,following_count,tweet_count,listed_count,coordinates,lat,long
124,es,1357494046369972224,2021-02-05 00:59:36+00:00,Ya me toca comprar otros briefs jajaja 😅.,1357494046369972224,0,everyone,Twitter for Android,False,002129bd2622e876,city,Colombia,"Neiva, Colombia",CO,NaN,"[-75.7061430026356, 2.71225299542781, -74.9383...",{},757040436129193984,https://pbs.twimg.com/profile_images/145133877...,gonorrreita,https://t.co/h0GSEfbHsQ,el maikel❤‍🔥,2016-07-24T02:31:20.000Z,False,False,el diario de un juguete,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,172,164,5800,0,"[3.216884690332782, -75.05316174871128]",[,3
134,es,1357494046369972224,2021-02-05 00:59:36+00:00,Ya me toca comprar otros briefs jajaja 😅.,1357494046369972224,0,everyone,Twitter for Android,False,002129bd2622e876,city,Colombia,"Neiva, Colombia",CO,NaN,"[-75.7061430026356, 2.71225299542781, -74.9383...",{},757040436129193984,https://pbs.twimg.com/profile_images/145133877...,gonorrreita,https://t.co/h0GSEfbHsQ,el maikel❤‍🔥,2016-07-24T02:31:20.000Z,False,False,el diario de un juguete,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,172,164,5800,0,"[3.2170298179049994, -74.51017496996857]",3.217029817904999422,-74.510174969968574032


In [19]:
t1 = df.iloc[:700000]
print(t1.shape)
t1['coordinates'] = t1['coordinates'].astype(str)

(700000, 37)


<ipython-input-19-b147501b48a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1['coordinates'] = t1['coordinates'].astype(str)


In [20]:
%%time
t = t1.groupby('id').nunique()[['coordinates', 'text']]
t.loc[t.coordinates > 1].sort_values('coordinates', ascending = False)

CPU times: user 5 s, sys: 68.5 ms, total: 5.07 s
Wall time: 5.07 s


,coordinates,text
id,,
1357494046369972224,3,2
1356220397704933376,2,1
1425439922274643968,2,1
1425432208316243968,2,1
1425432214989414400,2,1
...,...,...
1362967440968318976,2,1
1362967722406125568,2,1
1362968056742477824,2,1


In [21]:
df_dupli_sort = t1.drop_duplicates().sort_values('id')

In [22]:
%%time
t = df_dupli_sort.groupby('id').nunique()[['coordinates', 'text']]
t.loc[t.coordinates > 1].sort_values('coordinates', ascending = False)

CPU times: user 4.93 s, sys: 56.2 ms, total: 4.98 s
Wall time: 4.98 s


,coordinates,text
id,,
1357494046369972224,3,2
1356220397704933376,2,1
1425439922274643968,2,1
1425432208316243968,2,1
1425432214989414400,2,1
...,...,...
1362967440968318976,2,1
1362967722406125568,2,1
1362968056742477824,2,1


In [18]:
df_dupli_sort

,lang,id,created_at,text,conversation_id,in_reply_to_user_id,reply_settings,source,possibly_sensitive,geo_place_id,geo_place_type,geo_country,geo_full_name,geo_country_code,geo_geo_type,geo_geo_bbox,geo_geo_properties,author_id,author_profile_image_url,author_username,author_url,author_name,author_created_at,author_verified,author_protected,author_description,retweet_count,reply_count,like_count,quote_count,followers_count,following_count,tweet_count,listed_count,coordinates,lat,long
5623,es,1356210903507750912,2021-02-01 12:00:51+00:00,Todos esos barberos intensos me dan como ladil...,1356210903507750912,0,everyone,Twitter for Android,False,015e5397777769a2,city,Venezuela,"Mérida, Venezuela",VE,NaN,"[-71.1957474, 8.5660877, -71.1269677, 8.631216]",{},199494386,https://pbs.twimg.com/profile_images/144149718...,mierdatilin,NaN,siyei,2010-10-07T00:46:57.000Z,False,False,Pórtate bien,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,374,145,17710,0,"[8.635918710683876, -71.56215985722785]",[,8
2975,es,1356211246652149760,2021-02-01 12:02:13+00:00,Rusia prevé normas adicionales para los gigant...,1356211246652149760,0,everyone,Twitter for Android,True,00677a6154ac3141,city,Venezuela,"Juan Antonio Sotillo, Venezuela",VE,NaN,"[-64.6778108, 10.0977755, -64.5428047, 10.2360...",{},138189664,https://pbs.twimg.com/profile_images/144069332...,VencedorMRT,https://t.co/VeYKTQNNis,Roger Vásquez,2010-04-28T21:48:26.000Z,False,False,Técnico en Refinación. En lucha por la Igualda...,2.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,4937,4756,294965,31,"[10.267382711444606, -64.26246555082587]",[,1
3649,es,1356211458372235264,2021-02-01 12:03:03+00:00,@juliovreiker1 @mhcamargo @frankdurangoes @jor...,1356062666142396416,1280473987198443520,everyone,Twitter for Android,False,002129bd2622e876,city,Colombia,"Neiva, Colombia",CO,NaN,"[-75.7061430026356, 2.71225299542781, -74.9383...",{},128618233,https://pbs.twimg.com/profile_images/143678738...,jhsuarez71,NaN,JHSuárez,2010-04-01T17:29:45.000Z,False,False,"Se puede decir algo?... Ni lado ni color, solo...",0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,1417,1424,4159,0,"[3.216884690332782, -75.05316174871128]",[,3
2974,es,1356211545240444928,2021-02-01 12:03:24+00:00,Una exreina de belleza no puede conseguir trab...,1356211545240444928,0,everyone,Twitter for Android,True,00677a6154ac3141,city,Venezuela,"Juan Antonio Sotillo, Venezuela",VE,NaN,"[-64.6778108, 10.0977755, -64.5428047, 10.2360...",{},138189664,https://pbs.twimg.com/profile_images/144069332...,VencedorMRT,https://t.co/VeYKTQNNis,Roger Vásquez,2010-04-28T21:48:26.000Z,False,False,Técnico en Refinación. En lucha por la Igualda...,3.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,4937,4756,294965,31,"[10.267382711444606, -64.26246555082587]",[,1
2973,es,1356211650215473152,2021-02-01 12:03:49+00:00,Militares birmanos traspasarán el poder al gan...,1356211650215473152,0,everyone,Twitter for Android,True,00677a6154ac3141,city,Venezuela,"Juan Antonio Sotillo, Venezuela",VE,NaN,"[-64.6778108, 10.0977755, -64.5428047, 10.2360...",{},138189664,https://pbs.twimg.com/profile_images/144069332...,VencedorMRT,https://t.co/VeYKTQNNis,Roger Vásquez,2010-04-28T21:48:26.000Z,False,False,Técnico en Refinación. En lucha por la Igualda...,4.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,4937,4756,294965,31,"[10.267382711444606, -64.26246555082587]",[,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,es,1433036888609275904,2021-09-01 11:59:53+00:00,Necesito que la gente venga con un resumen de ...,1433036888609275904,0,everyone,Twitter for Android,False,0177894212b08f73,city,Argentina,"La Plata, Argentina",AR,NaN,"[-58.160874, -35.031373, -57.865169, -34.841264]",{},10294958787494871